Nearest-neighbor interpolation

Bilinear interpolation

Bicubic interpolation over 4x4 pixel neighborhood

Lanczos interpolation over 8x8 pixel neighborhood




Histogram equalization

Contrast-limited adaptive histogram equalization (CLAHE)

Gauss

Unsharp mask filtering

Median filtering

In [0]:
import cv2
import numpy
from google.colab.patches import cv2_imshow
from PIL import Image
import scipy.misc

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
source = "/content/drive/My Drive/Diploma_thesis/resized/"
path = "/content/drive/My Drive/Diploma_thesis/"

In [0]:
import shutil


def remove_dir(name):
  try:
      shutil.rmtree(name)
  except OSError:
      print ("Couldn't remove the directory")
  else:
      print ("Successfully removed the directory")

In [0]:
import os

def create_dir(name):
  #rm -rf name
  # define the name of the directory to be created
  try:
      os.mkdir(name)
  except OSError:
      print ("Creation of the directory failed")
  else:
      print ("Successfully created the directory")

In [0]:
def CLAHE(image, clip_limit=3):
    # convert image to LAB color model
    image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # split the image into L, A, and B channels
    l_channel, a_channel, b_channel = cv2.split(image_lab)

    # apply CLAHE to lightness channel
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)

    # merge the CLAHE enhanced L channel with the original A and B channel
    merged_channels = cv2.merge((cl, a_channel, b_channel))

    # convert iamge from LAB color model back to RGB color model
    final_image = cv2.cvtColor(merged_channels, cv2.COLOR_LAB2BGR)
    return final_image
    #Image.fromarray(cv2.cvtColor(final_image, cv2.COLOR_BGR2RGB))

In [0]:
def apply(inter_type, enh_type, img):
  res = img
  #interpolation
  if(inter_type == 'NN'):    
    res = cv2.resize(img, (0, 0), fx = 2, fy = 2, interpolation = cv2.INTER_NEAREST)
  elif(inter_type == 'Linear'):
    res = cv2.resize(img, (0, 0), fx = 2, fy = 2, interpolation = cv2.INTER_LINEAR)
  elif(inter_type == 'Bicubic'):
    res = cv2.resize(img, (0, 0), fx = 2, fy = 2, interpolation = cv2.INTER_CUBIC)
  elif(inter_type == 'Lanczos'):
    res = cv2.resize(img, (0, 0), fx = 2, fy = 2, interpolation = cv2.INTER_LANCZOS4)
    

  #enhancement
  if(enh_type == 'Equalization'):
    #img = cv2.equalizeHist(img)
    #R, G, B = cv2.split(res)

    #output1_R = cv2.equalizeHist(R)
    #output1_G = cv2.equalizeHist(G)
    #output1_B = cv2.equalizeHist(B)


    ycrcb=cv2.cvtColor(img,cv2.COLOR_BGR2YCR_CB)
    channels=cv2.split(ycrcb)
    cv2.equalizeHist(channels[0],channels[0])
    cv2.merge(channels,ycrcb)
    cv2.cvtColor(ycrcb,cv2.COLOR_YCR_CB2BGR,img)

    res = img
  elif(enh_type == 'CLAHE'):   
    res = CLAHE(res)
  elif(enh_type == 'Median'):
    res = cv2.medianBlur(res, 5)
  elif(enh_type == 'Gauss'):
    res = cv2.GaussianBlur(res, (5,5), 0)
  elif(enh_type == 'Unsharp'):
    #gaussian_3 = cv2.GaussianBlur(res, (9,9), 10.0)
    #res = cv2.addWeighted(res, 1.5, gaussian_3, -0.5, 0, res)
    print("Done")
    im_blurred = cv2.GaussianBlur(res, (11,11), 10)
    res = cv2.addWeighted(res, 1.0 + 3.0, im_blurred, -3.0, 0) 
  elif(enh_type == 'Average'):
    res = cv2.blur(img,(5,5))
  elif(enh_type == 'Sharpen'):
    kernel_sharpening = numpy.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
    res = cv2.filter2D(img, -1, kernel_sharpening)
  
  return res


In [35]:
#from zipfile import ZipFile

def main(interpolation, enhancement):
  listing = os.listdir(source) 

  dest = path + interpolation + '_' + enhancement 
  remove_dir(dest)
  create_dir(dest)

  #zipObj = ZipFile(dest + '.zip', 'w')

  for file in listing:
    img = cv2.imread(source + file)
    img = apply(interpolation, enhancement, img)
    name = file.split('.')
    cv2.imwrite(dest + '/' + name[0] + '.jpg', img)
    #zipObj.write(dest + '/' + file)

  

interpolation = 'None'
enhancement = 'Gauss'
dest = path + interpolation + '_' + enhancement
main(interpolation, enhancement)

Successfully removed the directory
Successfully created the directory


In [0]:
drive.flush_and_unmount()